In [2]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

import json
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /Users/billpark/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/billpark/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/billpark/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [4]:
# Function to load JSON data into a DataFrame
def load_json_to_df(json_file):
    with open(json_file, 'r') as file:
        data = json.load(file)
    return pd.DataFrame.from_dict(data, orient='index')

# List of JSON files
json_files = ['HR.json', 'HF.json', 'MR.json', 'MF.json']
dfs = []

# Load each JSON file into a DataFrame and store them in a list
for json_file in json_files:
    df = load_json_to_df('data/' + json_file)
    dfs.append(df)

# Naming the DataFrames
HR_df, HF_df, MR_df, MF_df = dfs

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

In [6]:
HR = HR_df['text']
HF = HF_df['text']
MR = MR_df['text']
MF = MF_df['text']

In [7]:
df = pd.concat([HR, HF, MR, MF], ignore_index=True)

In [38]:
from sklearn.metrics import accuracy_score, classification_report

# Assigning labels for each comparison
labels_hr_vs_mf = [0] * len(HR) + [1] * len(MF)
labels_hf_vs_mf = [0] * len(HF) + [1] * len(MF)
labels_mr_vs_mf = [0] * len(MR) + [1] * len(MF)
labels_hf_vs_mr = [0] * len(HF) + [1] * len(MR)
labels_hr_vs_mr = [0] * len(HR) + [1] * len(MR)
labels_hr_vs_hf = [0] * len(HR) + [1] * len(HF)

#Separate for Human vs AI in general
labels_hr_hf = [0] * (len(HR) + len(HF))
labels_mr_mf = [1] * (len(MR) + len(MF))

labels_human_machine = labels_hr_hf + labels_mr_mf



In [40]:
# Splitting the data for each comparison
X_train_hr_vs_mf, X_test_hr_vs_mf, y_train_hr_vs_mf, y_test_hr_vs_mf = train_test_split(pd.concat([HR, MF], ignore_index=True), labels_hr_vs_mf, test_size=0.2, random_state=42)
X_train_hf_vs_mf, X_test_hf_vs_mf, y_train_hf_vs_mf, y_test_hf_vs_mf = train_test_split(pd.concat([HF, MF], ignore_index=True), labels_hf_vs_mf, test_size=0.2, random_state=42)
X_train_mr_vs_mf, X_test_mr_vs_mf, y_train_mr_vs_mf, y_test_mr_vs_mf = train_test_split(pd.concat([MR, MF], ignore_index=True), labels_mr_vs_mf, test_size=0.2, random_state=42)
X_train_hf_vs_mr, X_test_hf_vs_mr, y_train_hf_vs_mr, y_test_hf_vs_mr = train_test_split(pd.concat([HF, MR], ignore_index=True), labels_hf_vs_mr, test_size=0.2, random_state=42)
X_train_hr_vs_mr, X_test_hr_vs_mr, y_train_hr_vs_mr, y_test_hr_vs_mr = train_test_split(pd.concat([HR, MR], ignore_index=True), labels_hr_vs_mr, test_size=0.2, random_state=42)
X_train_hr_vs_hf, X_test_hr_vs_hf, y_train_hr_vs_hf, y_test_hr_vs_hf = train_test_split(pd.concat([HR, HF], ignore_index=True), labels_hr_vs_hf, test_size=0.2, random_state=42)

# Splitting the data for the comparison between human-generated and machine-generated text
X_train_human_machine, X_test_human_machine, y_train_human_machine, y_test_human_machine = train_test_split(pd.concat([pd.concat([HR, HF]), pd.concat([MR, MF])], ignore_index=True),labels_human_machine,test_size=0.2,random_state=42)


In [42]:

# Creating TF-IDF vectorizer for each comparison
tfidf_vectorizer_hr_vs_mf = TfidfVectorizer(max_features=5000)
X_train_hr_vs_mf_tfidf = tfidf_vectorizer_hr_vs_mf.fit_transform(X_train_hr_vs_mf)
X_test_hr_vs_mf_tfidf = tfidf_vectorizer_hr_vs_mf.transform(X_test_hr_vs_mf)

tfidf_vectorizer_hf_vs_mf = TfidfVectorizer(max_features=5000)
X_train_hf_vs_mf_tfidf = tfidf_vectorizer_hf_vs_mf.fit_transform(X_train_hf_vs_mf)
X_test_hf_vs_mf_tfidf = tfidf_vectorizer_hf_vs_mf.transform(X_test_hf_vs_mf)

tfidf_vectorizer_mr_vs_mf = TfidfVectorizer(max_features=5000)
X_train_mr_vs_mf_tfidf = tfidf_vectorizer_mr_vs_mf.fit_transform(X_train_mr_vs_mf)
X_test_mr_vs_mf_tfidf = tfidf_vectorizer_mr_vs_mf.transform(X_test_mr_vs_mf)

tfidf_vectorizer_hf_vs_mr = TfidfVectorizer(max_features=5000)
X_train_hf_vs_mr_tfidf = tfidf_vectorizer_hf_vs_mr.fit_transform(X_train_hf_vs_mr)
X_test_hf_vs_mr_tfidf = tfidf_vectorizer_hf_vs_mr.transform(X_test_hf_vs_mr)

tfidf_vectorizer_hr_vs_mr = TfidfVectorizer(max_features=5000)
X_train_hr_vs_mr_tfidf = tfidf_vectorizer_hr_vs_mr.fit_transform(X_train_hr_vs_mr)
X_test_hr_vs_mr_tfidf = tfidf_vectorizer_hr_vs_mr.transform(X_test_hr_vs_mr)

tfidf_vectorizer_hr_vs_hf = TfidfVectorizer(max_features=5000)
X_train_hr_vs_hf_tfidf = tfidf_vectorizer_hr_vs_hf.fit_transform(X_train_hr_vs_hf)
X_test_hr_vs_hf_tfidf = tfidf_vectorizer_hr_vs_hf.transform(X_test_hr_vs_hf)

# human and machine in general

# Create TF-IDF vectorizer for human vs. machine comparison
tfidf_vectorizer_human_machine = TfidfVectorizer()
X_train_human_machine_tfidf = tfidf_vectorizer_human_machine.fit_transform(X_train_human_machine)
X_test_human_machine_tfidf = tfidf_vectorizer_human_machine.transform(X_test_human_machine)

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from xgboost import XGBClassifier
import pandas as pd

# Function to train and evaluate an XGBoost model
def train_and_evaluate_xgb(X_train, y_train, X_test, y_test):
    model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print("Accuracy:", accuracy)
    return model, predictions 

# Using the function to train and evaluate models for each comparison
def main():
    print("HR vs MF:")
    train_and_evaluate_xgb(X_train_hr_vs_mf_tfidf, y_train_hr_vs_mf, X_test_hr_vs_mf_tfidf, y_test_hr_vs_mf)
    print("HF vs MF:")
    train_and_evaluate_xgb(X_train_hf_vs_mf_tfidf, y_train_hf_vs_mf, X_test_hf_vs_mf_tfidf, y_test_hf_vs_mf)
    print("MR vs MF:")
    train_and_evaluate_xgb(X_train_mr_vs_mf_tfidf, y_train_mr_vs_mf, X_test_mr_vs_mf_tfidf, y_test_mr_vs_mf)
    print("HF vs MR:")
    train_and_evaluate_xgb(X_train_hf_vs_mr_tfidf, y_train_hf_vs_mr, X_test_hf_vs_mr_tfidf, y_test_hf_vs_mr)
    print("HR vs MR:")
    train_and_evaluate_xgb(X_train_hr_vs_mr_tfidf, y_train_hr_vs_mr, X_test_hr_vs_mr_tfidf, y_test_hr_vs_mr)
    print("HR vs HF:")
    train_and_evaluate_xgb(X_train_hr_vs_hf_tfidf, y_train_hr_vs_hf, X_test_hr_vs_hf_tfidf, y_test_hr_vs_hf)
    print("Human vs Machine:")
    train_and_evaluate_xgb(X_train_human_machine_tfidf, y_train_human_machine, X_test_human_machine_tfidf, y_test_human_machine)

if __name__ == "__main__":
    main()

HR vs MF:
Accuracy: 0.9726941747572816
HF vs MF:
Accuracy: 0.9675642594859241
MR vs MF:
Accuracy: 0.9485160508782556
HF vs MR:
Accuracy: 0.9133858267716536
HR vs MR:
Accuracy: 0.8672672672672672
HR vs HF:
Accuracy: 0.779126213592233
Human vs Machine:
Accuracy: 0.9238932686476653


In [29]:
from sklearn.metrics import confusion_matrix, accuracy_score

def display_results(X_test_tfidf, y_test, predictions, vectorizer):
    # Generate confusion matrix
    cm = confusion_matrix(y_test, predictions)
    print("Accuracy:", accuracy_score(y_test, predictions))
    print("Confusion Matrix:\n", cm)
    
    # Convert TF-IDF back to text
    X_test_texts = vectorizer.inverse_transform(X_test_tfidf)
    
    # Finding misclassified and correctly classified indices
    misclassified_indices = []
    correctly_classified_indices = []
    for i, (actual, predicted) in enumerate(zip(y_test, predictions)):
        if actual != predicted:
            misclassified_indices.append(i)
        else:
            correctly_classified_indices.append(i)
    
    # Display examples
    print("\nCorrectly Classified Examples:")
    for i in correctly_classified_indices[:3]:  # display first 3 correctly classified examples
        print(f"Text: {' '.join(X_test_texts[i])}\nLabel: {y_test[i]}\n")
    
    print("Misclassified Examples:")
    for i in misclassified_indices[:3]:  # display first 3 misclassified examples
        print(f"Text: {' '.join(X_test_texts[i])}\nActual Label: {y_test[i]}, Predicted Label: {predictions[i]}\n")

# # Example usage for one comparison:
# print("HR vs MF Examples:")
# model_hr_vs_mf, predictions_hr_vs_mf = train_and_evaluate_xgb(X_train_hr_vs_mf_tfidf, y_train_hr_vs_mf, X_test_hr_vs_mf_tfidf, y_test_hr_vs_mf)
# display_results(X_test_hr_vs_mf_tfidf, y_test_hr_vs_mf, predictions_hr_vs_mf, tfidf_vectorizer_hr_vs_mf)

### Visuals attempts

In [34]:
import matplotlib.pyplot as plt

def plot_xgb_history(results, comparison_name):
    epochs = range(len(results['validation_0']['logloss']))  # Assuming logloss and error are recorded
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, results['validation_0']['logloss'], label='Training Log Loss')
    plt.plot(epochs, results['validation_1']['logloss'], label='Validation Log Loss')
    plt.title('Log Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Log Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, results['validation_0']['error'], label='Training Error')
    plt.plot(epochs, results['validation_1']['error'], label='Validation Error')
    plt.title('Classification Error')
    plt.xlabel('Epoch')
    plt.ylabel('Error')
    plt.legend()

    plt.suptitle(f'Model Training and Validation Metrics for {comparison_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

In [37]:
plot_xgb_history(results_hr_vs_mf, "HR vs MF")

NameError: name 'results_hr_vs_mf' is not defined

### Word2Vec attemp from below

In [32]:
from gensim.models import Word2Vec

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')

def text_to_wordlist(text):
    return word_tokenize(text)

def build_word2vec_model(data_frames, text_extractor):
    sentences = []
    for df in data_frames:
        if isinstance(df, pd.DataFrame) and not df.empty:
            for _, row in df.iterrows():
                extracted_text = text_extractor(row)
                tokenized_text = text_to_wordlist(extracted_text)
                sentences.append(tokenized_text)
        else:
            print("Error: Provided data is not a DataFrame or is empty.")

    if sentences:
        model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
        return model
    else:
        raise ValueError("No sentences were extracted for Word2Vec training. Check your data and text extraction logic.")

def vectorize_with_word2vec(df, word2vec_model, text_extractor):
    def average_vector(row):
        text = text_extractor(row)
        words = text_to_wordlist(text)
        word_vectors = [word2vec_model.wv[word] for word in words if word in word2vec_model.wv]
        if len(word_vectors) == 0:
            return np.zeros(word2vec_model.vector_size)
        else:
            return np.mean(word_vectors, axis=0)
    return np.vstack(df.apply(average_vector, axis=1))

def prepare_and_vectorize_data(df1, df2, word2vec_model, text_extractor):
    labels = [0] * len(df1) + [1] * len(df2)
    X = pd.concat([df1, df2], ignore_index=True)
    X_vectors = vectorize_with_word2vec(X, word2vec_model, text_extractor)
    X_train, X_test, y_train, y_test = train_test_split(X_vectors, labels, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test

def train_and_evaluate_xgb(X_train, y_train, X_test, y_test, comparison_name):
    model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print(f"XGBoost Accuracy for {comparison_name}: {accuracy:.6f}")

# Assuming HR_df, HF_df, MR_df, MF_df are DataFrame objects and are loaded correctly
word2vec_model = build_word2vec_model([HR_df, HF_df, MR_df, MF_df], lambda row: extract_text(row, 'preprocessed_text', 'text'))

# Define pairs for comparison
pairs = [
    (HR_df, MF_df, "Human Real vs AI Fake"),
    (HF_df, MF_df, "Human Fake vs AI Fake"),
    (MR_df, MF_df, "AI Real vs AI Fake"),
    (HF_df, MR_df, "Human Fake vs AI Real"),
    (HR_df, MR_df, "Human Real vs AI Real"),
    (HR_df, HF_df, "Human Real vs Human Fake")
]

for df1, df2, name in pairs:
    X_train, X_test, y_train, y_test = prepare_and_vectorize_data(df1, df2, word2vec_model, lambda row: extract_text(row, 'preprocessed_text', 'text'))
    train_and_evaluate_xgb(X_train, y_train, X_test, y_test, name)
